In [27]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from wordcloud import WordCloud
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split


In [2]:
df_fake = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
df_True = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
for news in df_True.text.values:
    print(news)
    break

WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educati

In [6]:
df_True['Type']=1
df_fake['Type']=0

In [7]:
df_True.head()

,title,text,subject,date,Type
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
df_fake.head()

,title,text,subject,date,Type
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
df_True.drop(columns=['subject','date'],inplace = True)
df_fake.drop(columns=['subject','date'],inplace = True)

In [10]:
df_True['text']= df_True['title'] + " " + df_True['text']
df_fake['text']= df_fake['title'] + " " + df_fake['text']
df_True.drop(columns=['title'],inplace=True)
df_fake.drop(columns=['title'],inplace=True)

In [11]:
df_True.head()

,text,Type
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [12]:
df_True.shape, df_fake.shape

((21417, 2), (23481, 2))

In [13]:
df = pd.concat([df_True, df_fake], axis=0)
df.head()

,text,Type
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [14]:
df.shape

(44898, 2)

## Data cleaning

In [15]:
X = []
stop_words = set(nltk.corpus.stopwords.words("english"))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df["text"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
        tmp.extend(filtered_words)
    X.append(tmp)

In [25]:
X[30]

['alabama',
 'certify',
 'democrat',
 'jones',
 'winner',
 'senate',
 'election',
 'reuters',
 'democrat',
 'doug',
 'jones',
 'surprise',
 'victory',
 'republican',
 'roy',
 'moore',
 'month',
 'special',
 'senate',
 'election',
 'certified',
 'dec',
 '28',
 'alabama',
 'state',
 'officials',
 'said',
 'friday',
 'jones',
 'first',
 'democrat',
 'sent',
 'senate',
 'republican',
 'stronghold',
 'alabama',
 'quarter',
 'century',
 'takes',
 'office',
 'republicans',
 'majority',
 'chamber',
 'narrow',
 '51',
 '100',
 'seats',
 'alabama',
 'governor',
 'kay',
 'ivey',
 'attorney',
 'general',
 'steve',
 'marshall',
 'secretary',
 'state',
 'john',
 'merrill',
 'meet',
 'certify',
 'jones',
 'win',
 'merrill',
 'office',
 'said',
 'statement',
 'jones',
 'margin',
 'victory',
 'percentage',
 'points',
 'moore',
 'conceded',
 'defeat',
 'dec',
 '12',
 'vote',
 'despite',
 'urged',
 'president',
 'donald',
 'trump',
 'calls',
 'emails',
 'moore',
 'campaign',
 'spokeswomen',
 'immediately'

# Word Embedding 

## Word2Vec

In [18]:
import gensim

In [21]:
EMBEDDING_DIM = 100

w2v_model = gensim.models.Word2Vec(sentences=X, vector_size=EMBEDDING_DIM, window=5, min_count=1)

In [22]:
len(w2v_model.wv.index_to_key)

122369

In [26]:
w2v_model.wv.index_to_key[10]

'clinton'

In [29]:
x_train,x_test,y_train,y_test = train_test_split(X,df.Type,test_size =0.2,random_state = 42)

In [30]:
from keras.preprocessing import text

max_features = 10000

tokenizer = text.Tokenizer(num_words=max_features)

tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)

In [31]:
max=0
for i in range(len(tokenized_train)):
    if len(tokenized_train[i])> max:
        max= len(tokenized_train[i])
print(max)

4391


In [ ]:
from keras.utils import pad_sequences
x_train = pad_sequences(tokenized_train, maxlen=max)

tokenizer.fit_on_texts(x_test)
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = pad_sequences(tokenized_test, maxlen=maxlen)


In [ ]:
word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break    